### Empathetic Dialogues dataset fine-tuning

In [33]:
!pip install -U transformers
!pip install datasets
!pip install -U accelerate
!pip install tensorboard


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np # to get the predictions
from datasets import load_dataset
from transformers import TrainingArguments, Trainer

c:\Users\Diego\OneDrive\Desktop\empathAI\caldito\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
empathetic_dialogues_ds = load_dataset("Estwld/empathetic_dialogues_llm")
print(empathetic_dialogues_ds)

def extract_conversation_pairs(example):
    pairs = []
    for i in range(len(example['conversations']) - 1):
        if example['conversations'][i]['role'] == 'user' and example['conversations'][i + 1]['role'] == 'assistant':
            user_input = example['conversations'][i]['content']
            assistant_reponse = example['conversations'][i + 1]['content']
            pairs.append((user_input, assistant_reponse))
    return pairs

def preprocess_dataset(dataset):
    all_pairs = []
    for example in dataset:
        all_pairs.extend(extract_conversation_pairs(example))
    return all_pairs

train_pairs = preprocess_dataset(empathetic_dialogues_ds['train'])
eval_pairs = preprocess_dataset(empathetic_dialogues_ds['valid'])
print(train_pairs[0])
print(eval_pairs[0])

DatasetDict({
    train: Dataset({
        features: ['conv_id', 'situation', 'emotion', 'conversations'],
        num_rows: 19533
    })
    valid: Dataset({
        features: ['conv_id', 'situation', 'emotion', 'conversations'],
        num_rows: 2770
    })
    test: Dataset({
        features: ['conv_id', 'situation', 'emotion', 'conversations'],
        num_rows: 2547
    })
})
('I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world.', 'Was this a friend you were in love with, or just a best friend?')
('Today,as i was leaving for work in the morning,i had a tire burst in the middle of a busy road. That scared the hell out of me!', 'Are you fine now?')


In [3]:
# load the tokenizer and model
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [4]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import torch

tokenizer.pad_token = tokenizer.eos_token

# tokenize the dataset
def tokenize_pairs(pairs):
    inputs = []
    labels = []

    for user_input, assistant_response in pairs:
        encoded = tokenizer(
            user_input,
            assistant_response,
            truncation=True,
            padding='max_length',
            max_length=512,
            return_tensors='pt'
        )
        inputs.append(encoded['input_ids'])
        labels.append(encoded['attention_mask'])
    return {'input_ids': torch.cat(inputs), 'attention_mask': torch.cat(labels)}

train_data = tokenize_pairs(train_pairs)
eval_data = tokenize_pairs(eval_pairs)

In [6]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

# ensure the correct GPU is being used (if multiple are available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.tensor([1.0, 2.0, 3.0]).to(device)
print(x.device) # should print 'cuda:0'

True
1
NVIDIA GeForce RTX 4090
cuda:0


In [7]:
from torch.utils.data import Dataset

# define the custom Dataset class for processing dialog data
class DialogDataset(Dataset):
    def __init__(self, encodings):
        """
        initializes the DialogDataset with the encoded text data.

        :param encodings: a dictionary containing the encoded input data.
                           it should contain keys like 'input_ids' and 'attention_mask'
                           after tokenizing the raw text.
        """
        # store the encodings (tokenized inputs) passed to the class
        self.encodings = encodings

    def __len__(self):
        """ 
        returns the length of the dataset.

        the length is determined by the number of input sequences in the 'input_ids'
        of the encodings dictionary. all items (input sequences) should have the same length.

        :return: the number of samples in the dataset.
        """
        # return the number of samples, which is the same as the length of the 'input_ids' list
        return len(self.encodings['input_ids'])
        
    def __getitem__(self, idx):
        """
        retrieves a sample from the dataset at the specified index.

        the method returns the tokenized input for a given index (sample)
        in the dataset. the sample consists of input IDs and other related features
        (e.g., attention mask_mask) wrapped as a tensor.

        :param idx: the index of the sample to retrieve.
        :return: a dictionary where each key is a feature (e.g., 'input_ids', 'attention_mask')
        """
        # create and return a dictionary with each encoding key and its corresponding value
        # at the specified index. the values are converted into PyTorch tensors.
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = item['input_ids'].clone() # labels should be the same as input_ids
        return item
        
train_dataset = DialogDataset(train_data)
eval_dataset = DialogDataset(eval_data)

In [8]:
torch.cuda.empty_cache()

In [9]:
# model.save_pretrained("./trained_model")
# tokenizer.save_pretrained("./trained_model")

In [10]:
# metrics = trainer.evaluate()
# print(metrics)

In [11]:
from transformers import pipeline

# for generating predictions
chatbot = pipeline("text-generation", model="./trained_model", truncation=True, tokenizer=tokenizer)
response = chatbot("I'm feeling overwhelmed with everything going on in my life. Can you help me figure out how to deal with it? ", max_length=100)
print(response)

Device set to use cuda:0


[{'generated_text': "I'm feeling overwhelmed with everything going on in my life. Can you help me figure out how to deal with it? ive been trying to figure it out for the past few days."}]


In [12]:
# tokenizer = AutoTokenizer.from_pretrained("./trained_model")
# model = AutoModelForCausalLM.from_pretrained("./trained_model")

# # let's chat for 5 lines
# for step in range(5):
#     # encode the new user input, add the eos_token and return a tensor in Pytorch
#     new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

#     # append the new user input tokens to the chat history
#     bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

#     # generated a response while limiting the total chat history to 1000 tokens,
#     chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

#     # pretty print last output tokens from bot
#     print("EmpathAI: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

### Human-Like-DPO-Dataset Fine-tuning

In [13]:
# load the previously trained model
model_name = "./trained_model"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [14]:
import pandas as pd

# load the new dataset
human_like_dpo_ds = load_dataset("HumanLLMs/Human-Like-DPO-Dataset")
# convert the train split to a pandas DataFrame for easier manipulation
train_df = pd.DataFrame(human_like_dpo_ds['train'])
print(train_df.head())

                                              prompt  \
0   Oh, I just saw the best meme - have you seen it?   
1                   Do you have a go-to karaoke jam?   
2  **Crafty corner** Are you good at any DIY proj...   
3  What's your favorite type of cuisine to cook o...   
4              Do you have a secret talent or skill?   

                                              chosen  \
0  😂 Ah, no I haven't! I'm dying to know, what's ...   
1  Oh, totally! 😄 I'm a sucker for a good ol' roc...   
2  😊 I'm actually a big fan of DIY projects! I'm ...   
3  Oh, man! I'm a total sucker for Italian food! ...   
4  You know, I've always been fascinated by music...   

                                            rejected  
0  I'm an artificial intelligence language model,...  
1  As a professional AI language model, I don't h...  
2  Good day. As a continuously evolving artificia...  
3  In accordance with my programming, I must emph...  
4  Good day. As a professional AI language model,..

in the cell below, after tokenizing the text, i added the `labels` field, which will be used for training the model. the `labels` should be the same as the `input_ids` shifted by one token for casual language modeling.

In [15]:
# preprocess 
def preprocess_new_dataset(dataset, tokenizer):
    input_ids = []
    attention_masks = []
    labels = []

    for i in range(len(dataset["prompt"])):
        input_text = dataset["prompt"][i]
        target_text = dataset["chosen"][i]

        # format text into a tuple (prompt, chosen)
        formatted_text = f"prompt: {input_text},\nchosen: {target_text}"

        # tokenize formatted text
        tokenized_text = tokenizer(formatted_text, truncation=True, padding="max_length", max_length=512)

        # the labels are the same as the input_ids
        tokenized_text["labels"] = tokenized_text["input_ids"].copy()

        # append to lists
        input_ids.append(tokenized_text["input_ids"])
        attention_masks.append(tokenized_text["attention_mask"])
        labels.append(tokenized_text["labels"])

    # return a dictionary in the form expected by Dataset.from_dict()
    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "labels": labels
    }

processed_conversation = preprocess_new_dataset(train_df, tokenizer)
# debug: print first example
print(len(processed_conversation))
print(processed_conversation.keys())
print(processed_conversation["input_ids"][0])
print(tokenizer.decode(processed_conversation["input_ids"][0]))

3
dict_keys(['input_ids', 'attention_mask', 'labels'])
[16963, 457, 25, 3966, 11, 314, 655, 2497, 262, 1266, 25336, 532, 423, 345, 1775, 340, 21747, 198, 354, 5233, 25, 30325, 224, 7900, 11, 645, 314, 4398, 470, 0, 314, 1101, 9950, 284, 760, 11, 644, 338, 262, 25336, 546, 30, 1148, 340, 257, 8258, 3797, 393, 257, 11441, 3074, 30, 1338, 359, 262, 16567, 0, 12520, 97, 96, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 5025

after processing the dataset, it is converted back to a `Dataset` object for use in the `Trainer`:

In [16]:
from datasets import Dataset

human_like_convos_ds = Dataset.from_dict(processed_conversation)

In [58]:
# # define training arguments
# training_args = TrainingArguments(
#     output_dir="./human_like_dpo_results",             # directory to save model checkpoints
#     num_train_epochs=3,                 # adjust based on convergence
#     per_device_train_batch_size=4,      # increase to 16 if vram allows, or decrease to 8
#     per_device_eval_batch_size=8, 
#     gradient_accumulation_steps=4,      # helps if batch size is small
#     # evaluation_strategy="epoch",        # evaluate at the end of each epoch
#     save_strategy="epoch",              # save model checkpoints each epoch
#     logging_dir="./logs",               # logging directory
#     logging_steps=100,                  # adjust based on dataset size
#     learning_rate=5e-5,                 # starndard for transformer fine-tuning
#     warmup_steps=500,                   # helps stabilize training
#     weight_decay=0.01,                  # regularization
#     bf16=True,                          # enable mixed precision for speedup
#     save_total_limit=3,                 # keep last 3 checkpoints
#     # eval_accumulation_steps=4,          # accumuluate loss over multiple steps
#     logging_first_step=True,            # log first step
#     report_to="tensorboard",            # enable tensorboard integration
# )

# # create trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=human_like_convos_ds,
#     tokenizer=tokenizer,
# )

In [18]:
# # evaluate before fine-tuning
# pre_eval_results = trainer.evaluate(eval_dataset)

In [19]:
# print(pre_eval_results)

In [20]:
# torch.cuda.empty_cache()

In [21]:
# print(torch.cuda.is_available())
# print(torch.cuda.get_device_name())

In [22]:
# # fine-tune the model :)
# trainer.train(resume_from_checkpoint="./human_like_dpo_results/checkpoint-1360")

In [23]:
# model.save_pretrained("./human_like_dpo_model")
# tokenizer.save_pretrained("./human_like_dpo_model")

In [24]:
human_like_dpo_chatbot = pipeline("text-generation", model="./human_like_dpo_model", truncation=True, tokenizer=tokenizer)
response = chatbot("I'm feeling overwhelmed with everything going on in my life. Can you help me figure out how to deal with it? ", max_length=100)
print(response)

Device set to use cuda:0


[{'generated_text': "I'm feeling overwhelmed with everything going on in my life. Can you help me figure out how to deal with it? ive been trying to figure it out for the past few days."}]


In [25]:
# tokenizer = AutoTokenizer.from_pretrained("../empathAI/human_like_dpo_model")
# model = AutoModelForCausalLM.from_pretrained("../empathAI/human_like_dpo_model")

# # let's chat for 5 lines
# for step in range(5):
#     # encode the new user input, add the eos_token and return a tensor in Pytorch
#     new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

#     # append the new user input tokens to the chat history
#     bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

#     # generated a response while limiting the total chat history to 1000 tokens,
#     chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

#     # pretty print last output tokens from bot
#     print("EmpathAI: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

### Synthetic-persona-chat dataset fine-tuning

In [26]:
model_name = ".//human_like_dpo_model"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [27]:
synthetic_persona_chat_ds = load_dataset("google/Synthetic-Persona-Chat")
spc_train_df = pd.DataFrame(synthetic_persona_chat_ds["train"])
spc_eval_df = pd.DataFrame(synthetic_persona_chat_ds["validation"])
print(len(spc_train_df['user 1 personas']))
print(len(spc_train_df['user 2 personas']))
print(len(spc_train_df))

8938
8938
8938


In [28]:
print(spc_train_df.keys())
print(spc_train_df["Best Generated Conversation"][0])
print(spc_train_df['user 1 personas'][0])
print(spc_train_df['user 2 personas'][0])

Index(['user 1 personas', 'user 2 personas', 'Best Generated Conversation'], dtype='object')
User 1: Hi! I'm [user 1's name].
User 2: Hi [user 1's name], I'm [user 2's name].
User 1: What do you do for fun?
User 2: I like to play video games, go to the beach, and read.
User 1: I like to play video games too! I'm not much of a reader, though.
User 2: What video games do you like to play?
User 1: I like to play a lot of different games, but I'm really into competitive online games right now.
User 2: I'm not really into competitive games, I like to play more relaxing games.
User 1: That's cool. What kind of relaxing games do you like to play?
User 2: I like to play puzzle games, simulation games, and story-based games.
User 1: I've never been much of a puzzle game person, but I do like simulation games and story-based games.
User 2: Nice! What's your favorite simulation game?
User 1: I like Stardew Valley a lot. It's a farming game, but it's also really relaxing and fun.
User 2: I've hear

In [30]:
!pip install -U scikit-learn


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
# extract multiple exchanges from conversations
def extract_exchanges(example):
    all_exchanges = []

    user1_personas = example["user 1 personas"]
    user2_personas = example["user 2 personas"]
    bgc = example["Best Generated Conversation"]

    # outer loop: iterates through the whole conversation string and splits it into individual turns
    # inner loop: iterates through the list of strings and pairs each 'user 1' line with the next 'user 2' line
    for i in range(len(bgc)):
        conversation_lines = bgc[i].split("\n")
        conversation_pairs = []

        # create prompt-response pairs
        for j in range(0, len(conversation_lines)-1, 2):
            prompt = conversation_lines[j].replace("User 1: ", "")  # remove 'user 1' prefix
            response = conversation_lines[j+1].replace("User 2: ", "")  # remove 'user 2' prefix

            # persona of both users is included with each prompt-response pair
            conversation_pairs.append({
                "prompt": prompt,
                "response": response,
                "prompt_persona": user1_personas[i],
                "response_persona": user2_personas[i]
            })

        # add each converstaion pair to the main list that is to be returned
        all_exchanges.append({
            "user1_persona": user1_personas[i],
            "user2_persona": user2_personas[i],
            "conversation_pairs": conversation_pairs
        })

    return all_exchanges
        
all_exchanges = extract_exchanges(spc_train_df)
print(all_exchanges[0])

{'user1_persona': 'I am 32.\nI do not want a job.\nI play video games all day.\nI still live at home with my parents.', 'user2_persona': 'My favorite drink is iced coffee.\nI have a black belt in karate.\nI m in a jazz band and play the saxophone.\nI vacation along lake michigan every summer.', 'conversation_pairs': [{'prompt': "Hi! I'm [user 1's name].", 'response': "Hi [user 1's name], I'm [user 2's name].", 'prompt_persona': 'I am 32.\nI do not want a job.\nI play video games all day.\nI still live at home with my parents.', 'response_persona': 'My favorite drink is iced coffee.\nI have a black belt in karate.\nI m in a jazz band and play the saxophone.\nI vacation along lake michigan every summer.'}, {'prompt': 'What do you do for fun?', 'response': 'I like to play video games, go to the beach, and read.', 'prompt_persona': 'I am 32.\nI do not want a job.\nI play video games all day.\nI still live at home with my parents.', 'response_persona': 'My favorite drink is iced coffee.\nI 

In [77]:
# function for processing the conversations into a flat list of examples
def prepare_data_for_trainer(all_exchanges):
    examples = []

    for exchange in all_exchanges:
        for pair in exchange["conversation_pairs"]:
            if "prompt_persona" in pair and "response_persona" in pair:
                prompt_with_persona = f"Persona: {pair["prompt_persona"]}\nMessage: {pair["prompt"]}"
            else:
                prompt_with_persona = pair["prompt"]

            examples.append({
                "prompt": prompt_with_persona,
                "response": pair["response"]
            })

    return examples

# process the data
all_examples = prepare_data_for_trainer(all_exchanges)

In [53]:
# split the data
from sklearn.model_selection import train_test_split

train_examples, temp_examples = train_test_split(all_examples, test_size=0.2, random_state=42)
val_examples, test_examples = train_test_split(temp_examples, test_size=0.5, random_state=42)

In [56]:
# convert to Hugging Face Datasets
spc_train_dataset = Dataset.from_list(train_examples)
spc_val_dataset = Dataset.from_list(val_examples)
spc_test_dataset = Dataset.from_list(test_examples)

In [ ]:
# tokenization function
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # for autoregressive models like DialoGPT
    inputs = [f"{prompt} <|response|> {response} <|endoftext|>" 
              for prompt, response in zip(examples["prompt"], examples["response"])]
    
    result = tokenizer(inputs, padding="max_length", truncation=True, max_length=256)
    result["labels"] = result["input_ids"].copy()

    return result

# apply tokenization
tokenized_train = spc_train_dataset.map(tokenize_function, batched=True)
tokenized_val = spc_val_dataset.map(tokenize_function, batched=True)
tokenized_test = spc_test_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 12515/12515 [00:01<00:00, 12154.69 examples/s]


In [74]:
# define training arguments
training_args = TrainingArguments(
    output_dir="./spc_results",         # directory to save model checkpoints
    num_train_epochs=3,                 # adjust based on convergence
    per_device_train_batch_size=8,      # increase to 16 if vram allows, or decrease to 8
    per_device_eval_batch_size=8, 
    gradient_accumulation_steps=2,      # helps if batch size is small
    evaluation_strategy="epoch",        # evaluate at the end of each epoch
    save_strategy="epoch",              # save model checkpoints each epoch
    logging_dir="./logs/spc_logs",               # logging directory
    logging_steps=100,                  # adjust based on dataset size
    learning_rate=5e-5,                 # starndard for transformer fine-tuning
    warmup_steps=500,                   # helps stabilize training
    weight_decay=0.01,                  # regularization
    bf16=True,                          # enable mixed precision for speedup
    save_total_limit=3,                 # keep last 3 checkpoints
    eval_accumulation_steps=4,          # accumuluate loss over multiple steps
    logging_first_step=True,            # log first step
)

# create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

C:\Users\Diego\AppData\Local\Temp\ipykernel_14384\1116729622.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [61]:
# evaluate before fine-tuning
pre_eval_results = trainer.evaluate(eval_dataset)

C:\Users\Diego\AppData\Local\Temp\ipykernel_14384\967146244.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [62]:
print(pre_eval_results)

{'eval_loss': 0.24718457460403442, 'eval_model_preparation_time': 0.0094, 'eval_runtime': 54.4664, 'eval_samples_per_second': 105.331, 'eval_steps_per_second': 13.182}


In [72]:
torch.cuda.empty_cache()

print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
NVIDIA GeForce RTX 4090


In [75]:
# train the model :)
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.171200,0.159699
2,0.140500,0.145910


TrainOutput(global_step=18771, training_loss=0.1693690795175345, metrics={'train_runtime': 4388.447, 'train_samples_per_second': 68.441, 'train_steps_per_second': 4.277, 'total_flos': 1.394509101882409e+17, 'train_loss': 0.1693690795175345, 'epoch': 2.99960047942469})

In [76]:
model.save_pretrained("./spc_model")
tokenizer.save_pretrained("./spc_model")

('./spc_model\\tokenizer_config.json',
 './spc_model\\special_tokens_map.json',
 './spc_model\\vocab.json',
 './spc_model\\merges.txt',
 './spc_model\\added_tokens.json',
 './spc_model\\tokenizer.json')